## Setup

All libs that will be use in this note

In [1]:
import numpy as np
from glob import glob
import random
import time 

## Preprocessing

There are two folders that contains the inputs of the problem **Traveling Salesman Problem** they are different so let's split it.

### Test Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks/ED-TSP/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/ED-TSP


In [0]:
PATH_TEST = 'instancias/instancias_teste/'
DATA_TEST = glob(PATH_TEST+'*.txt')
print(DATA_TEST)

['instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/descricao.txt', 'instancias/instancias_teste/swiss42.txt']


### TSP Cup Data

In [0]:
PATH_CUP = 'instancias/instancias_tsp_cup/'
DATA_CUP = glob(PATH_CUP+'*.txt')
print(DATA_CUP)

['instancias/instancias_tsp_cup/descricao.txt', 'instancias/instancias_tsp_cup/tsp1.txt', 'instancias/instancias_tsp_cup/tsp2.txt', 'instancias/instancias_tsp_cup/tsp3.txt']


## Math

Some mathematics functions that will be useful 

In [0]:
def euclidian_distance(p1, p2):
    return int(np.linalg.norm(p1-p2).round())

In [0]:
print(euclidian_distance(np.array([1,1]), np.array([1,10])))

9


## Graph

We'll use a graph as data structure in order to representation our problem, where each city is a vertex and a connection is representation by a edge. 

In [0]:
class Graph:
    def __init__(self, num_vertices):
        self._adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=np.int64)
        
    def build(self, file, _type='tsp_test'):
        if _type == 'tsp_cup':
            self._handle_tsp_cup(file)
        elif _type == 'tsp_test':
            self._handle_tsp_test(file)
        else:
            print('Undefined type')
        
    def _handle_tsp_cup(self, file):
        data = []
        with open(file, 'r') as f:
            for line in f:
                try:
                    _, x, y = list(filter(lambda x: x!='', line.split(' ')))
                    data.append(np.array((float(x), float(y[:-1]))))
                except:
                    pass
        for i, p1 in enumerate(data):
            for j, p2 in enumerate(data):
                self._adjacency_matrix[i][j] = euclidian_distance(p1, p2)
    
    def _handle_tsp_test(self, file):
        data = []
        with open(file, 'r') as f:
            _input = f.readlines()
            _input = [_.strip() for _ in _input]
            _input = [_.split() for _ in _input]
            for i, array in enumerate(_input[3:]):
                try:
                    _ = [int(_) for _ in array]
                    if len(_) > 0:
                        data.append(_)
                except:
                    pass
        self._adjacency_matrix = data
    
    def num_vertices(self):
        return len(self._adjacency_matrix)
    
    def get_cost(self, _list_path_nodes):
        path = [self._adjacency_matrix[current][_next] for current, _next in zip(_list_path_nodes[1:], _list_path_nodes[:-1])]
        return sum(path)

## Optimizations Solutions

Some functions that will give us solutions for our problem.

## Helpers

Below functions that will be userful to generation solutions

In [0]:
def pick_element(_list, nodes):
    for index, x in enumerate(_list):
        if not nodes[index]:
            return index, x
    return None

In [0]:
def get_min_distance(_distances, nodes):
    min_index, min_val = pick_element(_distances, nodes)
    for index, x in enumerate(_distances):
        if min_val > x:
            min_val = x if not nodes[index] else min_val
            min_index = index if not nodes[index] else min_index
    return min_val, min_index

In [0]:
def get_cost_swap(graph, cost, _list, i, k):
    cost_swap = cost
    if i == k:
        return cost_swap
    if i > k:
        i, k = k, i
    if i+1 == k:
        prev, _next = i-1 if i > 0 else i, k+1 if k < graph.num_vertices() - 1 else k
        cost_swap -= graph.get_cost([_list[prev], _list[i], _list[k], _list[_next]])
        
        prev, _next = i-1 if i > 0 else k, k+1 if k < graph.num_vertices() - 1 else i
        cost_swap += graph.get_cost([_list[prev], _list[k], _list[i], _list[_next]])
    else:
        prev_i, _next_i = i-1 if i > 0 else i, i+1
        prev_k, _next_k = k-1, k+1 if k < graph.num_vertices() - 1 else k
        
        cost_swap -= graph.get_cost([_list[prev_i], _list[i], _list[_next_i]])
        cost_swap -= graph.get_cost([_list[prev_k], _list[k], _list[_next_k]])
        
        prev_i, _next_i = i-1 if i > 0 else k, i+1
        prev_k, _next_k = k-1, k+1 if k < graph.num_vertices() - 1 else i
        
        cost_swap += graph.get_cost([_list[prev_i], _list[k], _list[_next_i]])
        cost_swap += graph.get_cost([_list[prev_k], _list[i], _list[_next_k]])
    return cost_swap

### Random Restart

In [0]:
def random_restart(graph, itermax):
    _iter = 0
    solution = [i for i in range(graph.num_vertices())]
    np.random.shuffle(solution)
    current_solution, current_cost = solution, graph.get_cost(solution)
    while _iter < itermax:
        np.random.shuffle(solution)
        cost = graph.get_cost(solution)
        if current_cost > cost:
            current_cost, current_solution = cost, solution
            _iter = 0
        else:
            _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

### Nearest Neighbor

In [0]:
def nearest_neighbor(graph, initial):
    solution = []
    current_node = initial
    nodes = [False for x in range(graph.num_vertices())]
    nodes[current_node] = True
    flag = True
    solution.append(current_node)
    while flag:
        distances = graph._adjacency_matrix[current_node]
        min_distance, min_index = get_min_distance(distances, nodes)
        current_node = min_index
        nodes[current_node] = True
        solution.append(current_node)
        if all(nodes):
            solution.append(initial)
            flag = False
    return graph.get_cost(solution), solution

### Best Insertion

In [0]:
def best_insertion(graph, initial):
    current_cost, current_solution = initial
    index = np.random.randint(graph.num_vertices())
    for i in range(graph.num_vertices()):
        cost = get_cost_swap(graph, current_cost, current_solution, i, index)
        if current_cost > cost:
            current_solution[i], current_solution[index] = current_solution[index], current_solution[i]
            current_cost = cost
    return graph.get_cost(current_solution), current_solution

### Swap

In [0]:
def first_swap(graph, initial):
    current_cost, current_solution = initial
    flag = True
    while flag:
        flag = False
        for i in range(graph.num_vertices()):
            for k in range(i+1, graph.num_vertices()):
                cost = get_cost_swap(graph, current_cost, current_solution, i, k) 
                if current_cost > cost:
                    current_solution[i], current_solution[k] = current_solution[k], current_solution[i]
                    current_cost = cost
                    return graph.get_cost(current_solution), current_solution
    return graph.get_cost(current_solution), current_solution

### Perturbation

In [0]:
def perturbation(graph, _list):
    i=k=0
    while i==k:
        i, k = np.random.randint(graph.num_vertices()), np.random.randint(graph.num_vertices())
    if i > k:
        i, k = k, i
    start, middle, end = _list[:i], _list[i:k+1], _list[k+1:]
    middle.reverse()
    solution = start + middle + end
    return graph.get_cost(solution), solution

### VND - Variable Neighborhood Descent

In [0]:
def vnd(graph, initial, itermax):
    _iter = 0
    current_cost, current_solution = graph.get_cost(initial), initial
    while _iter < itermax:
        is_better = True
        while is_better:
            is_better = False
            cost, solution = best_insertion(graph, (current_cost, current_solution))
            if current_cost > cost:
                current_cost, current_solution = cost, solution
                is_better = True
                _iter = 0
            else:
                cost, solution = first_swap(graph, (current_cost, current_solution))
                if current_cost > cost:
                    current_cost, current_solution = cost, solution
                    is_better = True
                    _iter = 0
        _iter += 1
    return graph.get_cost(current_solution), current_solution

### GRASP - Greedy Randomized Adaptive Search Procedure 

In [0]:
def grasp(graph, itermax, alpha=0):
    _iter = 0
    current_cost = current_solution = -1
    while _iter < itermax:
        _random = np.random.random_sample() + alpha
        _, solution = nearest_neighbor(graph, np.random.randint(graph.num_vertices())) if _random < 1 else random_restart(graph, 100)
        solution = solution[:-1]
        cost, solution = vnd(graph, solution, 1)
        if current_cost > cost or _iter == 0:
            current_cost, current_solution = cost, solution
        _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

### VNS - Variable Neighborhood Search

In [2]:
def vns(graph, itermax):
    _iter = 0
    current_cost = current_solution = -1
    _, solution = nearest_neighbor(graph, np.random.randint(graph.num_vertices()))
    solution = solution[:-1]
    while _iter < itermax:
        for _ in range(4):
            _, solution = perturbation(graph, solution)
        cost, solution = vnd(graph, solution, 1)
        if current_cost > cost or _iter == 0:
            current_cost, current_solution = cost, solution
        _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

## Test

For each test we'll build a graph with the adjacency matrix representing our cities.

## Helpers

Below functions that will be userful to generation informations about our solutions

In [0]:
def gap(val_heur, optimal):
    return ((val_heur-optimal)/optimal)*100

In [0]:
def nearest_neighbor_test(graph, iter_max, optimal):
    indexes = []
    costs = []
    times = []
    random_index = np.random.randint(graph.num_vertices())
    indexes.append(random_index)
    _iter = 0
    while _iter < iter_max:
        while random_index in indexes:
            random_index = np.random.randint(graph.num_vertices())
        indexes.append(random_index)
        start = time.time()
        cost, solution = nearest_neighbor(graph, random_index)
        end = time.time()
        times.append(end-start)
        costs.append(cost)
        print('({:}) cost: {:} time: {:}'.format(_iter+1, cost, end-start))
        _iter += 1
    print('Mean Cost: {:} \t Optimal: {:} \t Mean Time: {:} \t Gap: {:}'
          .format(np.mean(costs), np.min(costs), np.mean(times), gap(np.min(costs),optimal)))

In [0]:
def grasp_test(graph, iter_max, optimal):
    _iter = 0
    costs = []
    times = []
    while _iter < iter_max:
        start = time.time()
        cost, solution = grasp(graph, 50, alpha=0.5)
        end = time.time()
        times.append(end-start)
        costs.append(cost)
        print('({:}) cost: {:} time: {:}'.format(_iter+1, cost, end-start))
        _iter += 1
    print('Mean Cost: {:} \t Optimal: {:} \t Mean Time: {:} \t Gap: {:}'
          .format(np.mean(costs), np.min(costs), np.mean(times), gap(np.min(costs),optimal)))

In [0]:
def vns_test(graph, iter_max, optimal):
    _iter = 0
    costs = []
    times = []
    while _iter < iter_max:
        start = time.time()
        cost, solution = vns(graph, 50)
        end = time.time()
        times.append(end-start)
        costs.append(cost)
        print('({:}) cost: {:} time: {:}'.format(_iter+1, cost, end-start))
        _iter += 1
    print('Mean Cost: {:} \t Optimal: {:} \t Mean Time: {:} \t Gap: {:}'
          .format(np.mean(costs), np.min(costs), np.mean(times), gap(np.min(costs),optimal)))

### TSP TEST

In [0]:
ITER_MAX = 10
print(DATA_TEST)

['instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/descricao.txt', 'instancias/instancias_teste/swiss42.txt']


### bayg29

In [0]:
graph = Graph(29)
graph.build(DATA_TEST[0], _type='tsp_test')
print(graph.num_vertices())

29


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 1610)

(1) cost: 2020 time: 0.00016069412231445312
(2) cost: 2110 time: 0.00012302398681640625
(3) cost: 2085 time: 0.00014019012451171875
(4) cost: 2263 time: 0.00011491775512695312
(5) cost: 1952 time: 0.00011801719665527344
(6) cost: 2052 time: 0.00011444091796875
(7) cost: 2149 time: 0.00011539459228515625
(8) cost: 2095 time: 0.00011229515075683594
(9) cost: 2014 time: 0.00011420249938964844
(10) cost: 2005 time: 0.0007457733154296875
Mean Cost: 2074.5 	 Optimal: 1952 	 Mean Time: 0.0001858949661254883 	 Gap: 21.242236024844722


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 1610)

(1) cost: 1889 time: 0.963158369064331
(2) cost: 1969 time: 0.9000062942504883
(3) cost: 1969 time: 0.8635907173156738
(4) cost: 1969 time: 0.98714280128479
(5) cost: 1897 time: 0.9855978488922119
(6) cost: 1969 time: 0.8711667060852051
(7) cost: 1897 time: 0.896195650100708
(8) cost: 1872 time: 0.9983537197113037
(9) cost: 1897 time: 1.0292608737945557
(10) cost: 1897 time: 1.028688907623291
Mean Cost: 1922.5 	 Optimal: 1872 	 Mean Time: 0.9523161888122559 	 Gap: 16.273291925465838


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 1610)

(1) cost: 1796 time: 0.5350902080535889
(2) cost: 1953 time: 0.5878357887268066
(3) cost: 1749 time: 0.5455679893493652
(4) cost: 1964 time: 0.5881626605987549
(5) cost: 1760 time: 0.6722347736358643
(6) cost: 1897 time: 0.48314499855041504
(7) cost: 1777 time: 0.6087136268615723
(8) cost: 1756 time: 0.6632716655731201
(9) cost: 1825 time: 0.5968742370605469
(10) cost: 1990 time: 0.6508467197418213
Mean Cost: 1846.7 	 Optimal: 1749 	 Mean Time: 0.5931742668151856 	 Gap: 8.633540372670808


### bays29

In [0]:
graph = Graph(29)
graph.build(DATA_TEST[1], _type='tsp_test')
print(graph.num_vertices())

29


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 2020)

(1) cost: 2588 time: 0.00016450881958007812
(2) cost: 2375 time: 0.00016307830810546875
(3) cost: 2332 time: 0.00014829635620117188
(4) cost: 2356 time: 0.0001361370086669922
(5) cost: 2134 time: 0.000148773193359375
(6) cost: 2356 time: 0.00014328956604003906
(7) cost: 2411 time: 0.0001556873321533203
(8) cost: 2380 time: 0.00016045570373535156
(9) cost: 2293 time: 0.00015974044799804688
(10) cost: 2382 time: 0.00023412704467773438
Mean Cost: 2360.7 	 Optimal: 2134 	 Mean Time: 0.0001614093780517578 	 Gap: 5.643564356435644


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 2020)

(1) cost: 2035 time: 0.9491374492645264
(2) cost: 2255 time: 0.8849806785583496
(3) cost: 2433 time: 0.8431110382080078
(4) cost: 2255 time: 0.9915552139282227
(5) cost: 2035 time: 1.0158419609069824
(6) cost: 2255 time: 0.9508662223815918
(7) cost: 2255 time: 0.938401460647583
(8) cost: 2120 time: 0.86014723777771
(9) cost: 2255 time: 0.9875378608703613
(10) cost: 2035 time: 0.9686813354492188
Mean Cost: 2193.3 	 Optimal: 2035 	 Mean Time: 0.9390260457992554 	 Gap: 0.7425742574257426


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 2020)

(1) cost: 2209 time: 0.4732084274291992
(2) cost: 2200 time: 0.5364894866943359
(3) cost: 2402 time: 0.6147122383117676
(4) cost: 2386 time: 0.6009609699249268
(5) cost: 2415 time: 0.5392060279846191
(6) cost: 2477 time: 0.48884081840515137
(7) cost: 2306 time: 0.525183916091919
(8) cost: 2086 time: 0.5677998065948486
(9) cost: 2322 time: 0.6451361179351807
(10) cost: 2228 time: 0.5789871215820312
Mean Cost: 2303.1 	 Optimal: 2086 	 Mean Time: 0.557052493095398 	 Gap: 3.2673267326732676


### berlin52

In [0]:
graph = Graph(52)
graph.build(DATA_TEST[2], _type='tsp_test')
print(graph.num_vertices())

52


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 7542)

(1) cost: 9073 time: 0.00034236907958984375
(2) cost: 9504 time: 0.0003178119659423828
(3) cost: 9771 time: 0.0005443096160888672
(4) cost: 9897 time: 0.0005371570587158203
(5) cost: 9461 time: 0.0004482269287109375
(6) cost: 9257 time: 0.0005652904510498047
(7) cost: 9334 time: 0.00042319297790527344
(8) cost: 9098 time: 0.0004277229309082031
(9) cost: 9123 time: 0.0007660388946533203
(10) cost: 9192 time: 0.0004062652587890625
Mean Cost: 9371.0 	 Optimal: 9073 	 Mean Time: 0.0004778385162353516 	 Gap: 20.29965526385574


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 7542)

(1) cost: 8156 time: 4.372938871383667
(2) cost: 8156 time: 4.689905405044556
(3) cost: 8156 time: 4.55724835395813
(4) cost: 8156 time: 4.4663543701171875
(5) cost: 9009 time: 5.816087245941162
(6) cost: 9009 time: 4.19988751411438
(7) cost: 9009 time: 5.385004997253418
(8) cost: 8156 time: 4.510770797729492
(9) cost: 8156 time: 4.590519428253174
(10) cost: 8156 time: 4.319025039672852
Mean Cost: 8411.9 	 Optimal: 8156 	 Mean Time: 4.690774202346802 	 Gap: 8.141076637496687


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 7542)

(1) cost: 8789 time: 2.7834420204162598
(2) cost: 9166 time: 2.725464105606079
(3) cost: 9314 time: 2.945899248123169
(4) cost: 7949 time: 2.864553213119507
(5) cost: 9021 time: 2.9881703853607178
(6) cost: 9047 time: 2.888876438140869
(7) cost: 8641 time: 3.095512628555298
(8) cost: 10183 time: 3.0002989768981934
(9) cost: 8573 time: 2.6782429218292236
(10) cost: 9098 time: 2.4113693237304688
Mean Cost: 8978.1 	 Optimal: 7949 	 Mean Time: 2.8381829261779785 	 Gap: 5.39644656589764


### bier127

In [0]:
graph = Graph(127)
graph.build(DATA_TEST[3], _type='tsp_test')
print(graph.num_vertices())

127


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 118282)

(1) cost: 149525 time: 0.0020253658294677734
(2) cost: 137572 time: 0.001787424087524414
(3) cost: 141595 time: 0.0020520687103271484
(4) cost: 150310 time: 0.004904031753540039
(5) cost: 148330 time: 0.004082441329956055
(6) cost: 156561 time: 0.0017611980438232422
(7) cost: 141487 time: 0.0021576881408691406
(8) cost: 151056 time: 0.0015006065368652344
(9) cost: 149303 time: 0.001720428466796875
(10) cost: 150915 time: 0.0019979476928710938
Mean Cost: 147665.4 	 Optimal: 137572 	 Mean Time: 0.0023989200592041014 	 Gap: 16.308483116619605


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 118282)

(1) cost: 131515 time: 70.23261833190918
(2) cost: 135042 time: 84.86787104606628
(3) cost: 129630 time: 72.11058378219604
(4) cost: 129630 time: 75.83161926269531
(5) cost: 130323 time: 69.55900764465332
(6) cost: 133617 time: 85.15590572357178
(7) cost: 134535 time: 77.8014657497406
(8) cost: 135097 time: 74.88356423377991
(9) cost: 130323 time: 86.66766953468323
(10) cost: 129630 time: 67.606853723526
Mean Cost: 131934.2 	 Optimal: 129630 	 Mean Time: 76.47171590328216 	 Gap: 9.594021068294415


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 118282)

(1) cost: 141194 time: 25.430795907974243
(2) cost: 143691 time: 28.266698598861694
(3) cost: 139735 time: 24.295819520950317
(4) cost: 147875 time: 24.45282244682312
(5) cost: 139448 time: 26.267772674560547
(6) cost: 147204 time: 25.31448745727539
(7) cost: 142494 time: 25.731751441955566
(8) cost: 140497 time: 27.34796714782715
(9) cost: 152720 time: 26.12026309967041
(10) cost: 161817 time: 22.48552680015564
Mean Cost: 145667.5 	 Optimal: 139448 	 Mean Time: 25.571390509605408 	 Gap: 17.8945232579767


### brazil58

In [0]:
graph = Graph(58)
graph.build(DATA_TEST[4], _type='tsp_test')
print(graph.num_vertices())

58


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 25395)

(1) cost: 29168 time: 0.0005629062652587891
(2) cost: 27384 time: 0.0004954338073730469
(3) cost: 30032 time: 0.0004661083221435547
(4) cost: 32104 time: 0.0005066394805908203
(5) cost: 30774 time: 0.0007119178771972656
(6) cost: 27384 time: 0.0006091594696044922
(7) cost: 32619 time: 0.0011272430419921875
(8) cost: 32856 time: 0.0005536079406738281
(9) cost: 31836 time: 0.0004494190216064453
(10) cost: 31931 time: 0.0003414154052734375
Mean Cost: 30608.8 	 Optimal: 27384 	 Mean Time: 0.0005823850631713867 	 Gap: 7.832250443000591


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 25395)

(1) cost: 27471 time: 7.11462664604187
(2) cost: 27826 time: 5.862435579299927
(3) cost: 27471 time: 7.67265510559082
(4) cost: 27826 time: 6.27186393737793
(5) cost: 27477 time: 7.047261714935303
(6) cost: 26839 time: 5.6522016525268555
(7) cost: 27477 time: 7.337691307067871
(8) cost: 27160 time: 6.099673748016357
(9) cost: 27249 time: 6.4857401847839355
(10) cost: 26840 time: 7.686581611633301
Mean Cost: 27363.6 	 Optimal: 26839 	 Mean Time: 6.723073148727417 	 Gap: 5.686158692656035


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 25395)

(1) cost: 30476 time: 3.232343912124634
(2) cost: 29135 time: 3.699176788330078
(3) cost: 33466 time: 4.16161847114563
(4) cost: 31036 time: 3.548258066177368
(5) cost: 35601 time: 3.696650505065918
(6) cost: 29606 time: 3.83724045753479
(7) cost: 30417 time: 3.6443750858306885
(8) cost: 27125 time: 4.026474952697754
(9) cost: 31672 time: 3.9775238037109375
(10) cost: 28855 time: 3.1570236682891846
Mean Cost: 30738.9 	 Optimal: 27125 	 Mean Time: 3.6980685710906984 	 Gap: 6.812364638708407


### ch130

In [0]:
graph = Graph(130)
graph.build(DATA_TEST[5], _type='tsp_test')
print(graph.num_vertices())

130


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 6110)

(1) cost: 7684 time: 0.0016741752624511719
(2) cost: 7529 time: 0.0020575523376464844
(3) cost: 8015 time: 0.003407716751098633
(4) cost: 7379 time: 0.003601551055908203
(5) cost: 7642 time: 0.0017371177673339844
(6) cost: 8081 time: 0.0015225410461425781
(7) cost: 7580 time: 0.0014858245849609375
(8) cost: 7228 time: 0.001424551010131836
(9) cost: 7779 time: 0.0014383792877197266
(10) cost: 7229 time: 0.0014300346374511719
Mean Cost: 7614.6 	 Optimal: 7228 	 Mean Time: 0.0019779443740844727 	 Gap: 18.29787234042553


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 6110)

(1) cost: 6917 time: 74.55113053321838
(2) cost: 7014 time: 66.05172395706177
(3) cost: 7014 time: 86.56622576713562
(4) cost: 7151 time: 80.32587575912476
(5) cost: 7197 time: 77.2392680644989
(6) cost: 7197 time: 76.05903792381287
(7) cost: 7317 time: 86.77890157699585
(8) cost: 7014 time: 71.12981486320496
(9) cost: 7014 time: 66.05755090713501
(10) cost: 7197 time: 71.91236162185669
Mean Cost: 7103.2 	 Optimal: 6917 	 Mean Time: 75.66718909740447 	 Gap: 13.20785597381342


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 6110)

(1) cost: 7358 time: 27.042497634887695
(2) cost: 8996 time: 24.71938705444336
(3) cost: 7588 time: 29.595391511917114
(4) cost: 7024 time: 24.032722234725952
(5) cost: 7629 time: 23.066632747650146
(6) cost: 8706 time: 24.73310422897339
(7) cost: 8236 time: 29.461508750915527
(8) cost: 8012 time: 27.467189073562622
(9) cost: 8048 time: 23.29631209373474
(10) cost: 7240 time: 26.832247734069824
Mean Cost: 7883.7 	 Optimal: 7024 	 Mean Time: 26.02469930648804 	 Gap: 14.959083469721769


### ch150

In [0]:
graph = Graph(150)
graph.build(DATA_TEST[6], _type='tsp_test')
print(graph.num_vertices())

150


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 6528)

(1) cost: 7679 time: 0.002885580062866211
(2) cost: 7627 time: 0.005655765533447266
(3) cost: 7437 time: 0.0021584033966064453
(4) cost: 7260 time: 0.0022516250610351562
(5) cost: 7260 time: 0.0024442672729492188
(6) cost: 8031 time: 0.002015829086303711
(7) cost: 7559 time: 0.002248525619506836
(8) cost: 7466 time: 0.0023050308227539062
(9) cost: 7572 time: 0.0019989013671875
(10) cost: 7717 time: 0.0019598007202148438
Mean Cost: 7560.8 	 Optimal: 7260 	 Mean Time: 0.0025923728942871095 	 Gap: 11.213235294117647


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 6528)

(1) cost: 6977 time: 128.53618478775024
(2) cost: 7097 time: 116.54495620727539
(3) cost: 7208 time: 113.57432866096497
(4) cost: 7293 time: 105.4838445186615
(5) cost: 7097 time: 127.18448758125305
(6) cost: 7097 time: 106.78445506095886
(7) cost: 7276 time: 116.50320625305176
(8) cost: 7276 time: 125.38474822044373
(9) cost: 6964 time: 127.23568940162659
(10) cost: 7276 time: 107.65196442604065
Mean Cost: 7156.1 	 Optimal: 6964 	 Mean Time: 117.48838651180267 	 Gap: 6.678921568627451


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 6528)

(1) cost: 8303 time: 45.805487394332886
(2) cost: 8060 time: 33.94316816329956
(3) cost: 7901 time: 29.789414167404175
(4) cost: 7792 time: 42.847081422805786
(5) cost: 7657 time: 35.90589880943298
(6) cost: 7768 time: 41.244534969329834
(7) cost: 7707 time: 35.58925724029541
(8) cost: 7951 time: 30.932342767715454
(9) cost: 7537 time: 32.385844707489014
(10) cost: 8469 time: 36.01864266395569
Mean Cost: 7914.5 	 Optimal: 7537 	 Mean Time: 36.44616723060608 	 Gap: 15.456495098039216


### swiss42

In [0]:
graph = Graph(42)
graph.build(DATA_TEST[8], _type='tsp_test')
print(graph.num_vertices())

42


#### Nearest Neighbor

In [0]:
nearest_neighbor_test(graph, ITER_MAX, 1273)

(1) cost: 1577 time: 0.00029540061950683594
(2) cost: 1469 time: 0.00026702880859375
(3) cost: 1701 time: 0.0002777576446533203
(4) cost: 1564 time: 0.00026917457580566406
(5) cost: 1547 time: 0.0002722740173339844
(6) cost: 1648 time: 0.00030922889709472656
(7) cost: 1786 time: 0.000286102294921875
(8) cost: 1561 time: 0.00031113624572753906
(9) cost: 1549 time: 0.0002911090850830078
(10) cost: 1648 time: 0.00033164024353027344
Mean Cost: 1605.0 	 Optimal: 1469 	 Mean Time: 0.0002910852432250977 	 Gap: 15.396700706991359


#### Greedy Randomized Adaptive Search Procedure

In [0]:
grasp_test(graph, ITER_MAX, 1273)

(1) cost: 1514 time: 2.6217150688171387
(2) cost: 1313 time: 2.647536516189575
(3) cost: 1337 time: 2.719986915588379
(4) cost: 1313 time: 2.368317127227783
(5) cost: 1516 time: 2.5499539375305176
(6) cost: 1537 time: 2.5619969367980957
(7) cost: 1313 time: 2.2339351177215576
(8) cost: 1313 time: 2.091729164123535
(9) cost: 1313 time: 2.4259517192840576
(10) cost: 1313 time: 2.58292293548584
Mean Cost: 1378.2 	 Optimal: 1313 	 Mean Time: 2.4804045438766478 	 Gap: 3.1421838177533385


#### Variable Neighborhood Search

In [0]:
vns_test(graph, ITER_MAX, 1273)

(1) cost: 1428 time: 1.4467768669128418
(2) cost: 1480 time: 1.4454796314239502
(3) cost: 1519 time: 1.65087890625
(4) cost: 1366 time: 1.4317922592163086
(5) cost: 1514 time: 1.4700894355773926
(6) cost: 1606 time: 1.2753872871398926
(7) cost: 1313 time: 1.6444714069366455
(8) cost: 1577 time: 1.488330364227295
(9) cost: 1654 time: 1.4007072448730469
(10) cost: 1490 time: 1.4989776611328125
Mean Cost: 1494.7 	 Optimal: 1313 	 Mean Time: 1.4752891063690186 	 Gap: 3.1421838177533385
